In [ ]:
#https://github.com/yogesh-kamat/EduMeet.git
#used above github repository as reference

In [10]:
import argparse
from pathlib import Path
from PIL import Image
import numpy as np
from numpy import asarray
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense,Embedding,LSTM,SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import RMSprop

In [ ]:
testframes='t2frames'
trainframes='tframes'
validationframes='valframes'

#loading trainframes
x = np.load(f'x_{trainframes}.npy', allow_pickle=True)
y = np.load(f'y_{trainframes}.npy')
dataset = tf.data.Dataset.from_tensor_slices(
        (x, {"y1": y[:, :1], "y2": y[:, 1:2],
             "y3": y[:, 2:3], "y4": y[:, :3:4]}))

#loading validationframes
u = np.load(f'x_{validationframes}.npy', allow_pickle=True)
v = np.load(f'y_{validationframes}.npy')
dataset_val = tf.data.Dataset.from_tensor_slices(
        (u, {"y1": v[:, :1], "y2": v[:, 1:2],
             "y3": v[:, 2:3], "y4": v[:, :3:4]}))

#loading testframes
w = np.load(f'x_{testframes}.npy', allow_pickle=True)
z = np.load(f'y_{testframes}.npy')
dataset_test = tf.data.Dataset.from_tensor_slices(
        (w, {"y1": z[:, :1], "y2": z[:, 1:2],
             "y3": z[:, 2:3], "y4": z[:, :3:4]}))

def mapping(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(contents=img, channels=3)
    img = tf.image.convert_image_dtype(image=img, dtype=tf.float32)
    img = tf.image.resize(images=img,
                            size=[img_width, img_height],
                            method=tf.image.ResizeMethod.BILINEAR,
                            antialias=True)
    return img, label

dataset = dataset.map(map_func=mapping, num_parallel_calls=autotune)
dataset_val = dataset_val.map(map_func=mapping, num_parallel_calls=autotune)
dataset_test = dataset_test.map(map_func=mapping, num_parallel_calls=autotune)

autotune = tf.data.experimental.AUTOTUNE
img_width = 299
img_height = 299
batch_size = 16
shuffle_buffer_size = 2000

dataset = dataset.shuffle(buffer_size=shuffle_buffer_size,
                          reshuffle_each_iteration=True)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(autotune)

dataset_val = dataset_val.shuffle(buffer_size=shuffle_buffer_size,
                          reshuffle_each_iteration=True)
dataset_val = dataset_val.batch(batch_size)
dataset_val = dataset_val.prefetch(autotune)

dataset_test = dataset_test.shuffle(buffer_size=shuffle_buffer_size,
                          reshuffle_each_iteration=True)
dataset_test = dataset_test.batch(batch_size)
dataset_test = dataset_test.prefetch(autotune)

base_model = keras.applications.VGG16(
    weights='imagenet',  
    input_shape=(299, 299, 3),
    include_top=False)  
base_model.trainable = False

classes=4

inputs = keras.Input(shape=(299, 299, 3))

x1 = base_model(inputs, training=False)
x1 = keras.layers.GlobalAveragePooling2D()(x1)
initializer = tf.keras.initializers.GlorotUniform(seed=42)

boredom = Dense(4, name="y1")(x1)
engagement = Dense(4, name="y2")(x1)
confusion = Dense(4, name="y3")(x1)
frustration = Dense(4, name="y4")(x1)
# model = Model(inputs=base_model.input,
#                       outputs=[boredom, engagement, confusion, frustration])  
model = keras.Model(inputs,  outputs=[boredom, engagement, confusion, frustration])

model.compile(optimizer=keras.optimizers.Adam(0.0001),
loss={"y1": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        "y2": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        "y3": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        "y4": keras.losses.SparseCategoricalCrossentropy(from_logits=True)},
                  metrics={"y1": "sparse_categorical_accuracy",
                           "y2": "sparse_categorical_accuracy",
                           "y3": "sparse_categorical_accuracy",
                           "y4": "sparse_categorical_accuracy"})

model.fit(dataset,epochs=5,initial_epoch=0,validation_data=dataset_val)

model.evaluate(dataset_test)